In [ ]:
import requests, csv
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/catalogue/page-1.html"
data = []

for _ in range(5):   # number of pages
    soup = BeautifulSoup(requests.get(url).text, "html.parser")

    for b in soup.select(".product_pod"):
        title = b.h3.a["title"]
        price = b.select_one(".price_color").text
        avail = b.select_one(".availability").text.strip()
        link = "https://books.toscrape.com/catalogue/" + b.h3.a["href"]
        data.append([title, price, avail, link])

    nxt = soup.select_one(".next a")
    if not nxt: break
    url = "https://books.toscrape.com/catalogue/" + nxt["href"]

with open("products.csv", "w", newline="", encoding="utf-8") as f:
    csv.writer(f).writerows([("Title","Price","Availability","Link")] + data)

print("Saved:", len(data), "items")


Crawling page: https://books.toscrape.com/catalogue/page-1.html
Crawling page: https://books.toscrape.com/catalogue/page-2.html
Crawling page: https://books.toscrape.com/catalogue/page-3.html
Crawling page: https://books.toscrape.com/catalogue/page-4.html
Crawling page: https://books.toscrape.com/catalogue/page-5.html
Crawling page: https://books.toscrape.com/catalogue/page-6.html
Crawling page: https://books.toscrape.com/catalogue/page-7.html
Crawling page: https://books.toscrape.com/catalogue/page-8.html
Crawling page: https://books.toscrape.com/catalogue/page-9.html
Crawling page: https://books.toscrape.com/catalogue/page-10.html
Saved 200 products to books.csv
